In [7]:
import os

In [9]:
os.environ["SPARK_HOME"]

'/home/besgam/anaconda3/bin'

In [3]:
os.environ["SPARK_HOME"] = "/home/besgam/anaconda3/bin"
os.environ["PYSPARK_PYTHON"]="/home/besgam/anaconda3/bin/python3.8"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pytrends.request import TrendReq

spark = SparkSession.builder.appName("Google Trends") \
    .master("spark://localhost:7077")\
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")\
    .config("hive.exe.dynamic.partition.mode", "nonstrict") \
    .getOrCreate()

from delta.tables import *

In [ ]:
pytrends = TrendReq(hl='pt-BR', tz=-300)

In [ ]:
kw_list = ["Santander", "Banco"]
pytrends.build_payload(kw_list, timeframe='now 1-H', geo='BR')

In [ ]:
df = pytrends.related_queries()

In [ ]:
mySchema = StructType([ StructField("query", StringType(), True)\
                       ,StructField("value", IntegerType(), True)])

In [ ]:
csvDF = spark \
    .readStream \
    .format("csv")
    .option("sep", ",") \
    .schema(mySchema) \
    .load()

In [ ]:
sparkDF2 = spark.createDataFrame(df['Santander']['top'],schema=mySchema)

In [ ]:
sparkDF2.show(100,False)

In [33]:
type(sparkDF2)

pyspark.sql.dataframe.DataFrame

In [ ]:
sparkDF2.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [31]:
df['Santander']['rising']